## Backtesting trading strategies

In [1]:
import pandas as pd 
import yfinance as yf
import sys
sys.path.append('../src')
from data import load_data
from strategies.macd import MACD
from strategies.stock_trend_following_system import StockTrend
from parallel_processing import ParallelProcess
import talib
import numpy as np
import dask
import multiprocessing
from helper import Profit
from strategies.golden_death_cross import GoldenDeathCross

sys.path.append("../")
import conf

# instantiate config
STRATEGY = conf.backtest_conf['strategy']

%load_ext autoreload
%autoreload 2

### EDA on preprocessed dataset
1. Size of data
2. Missing values

In [2]:
# Load preprocessed data 
PREPROCESSED_DATAPATH = conf.backtest_conf["data"]["preprocessed_datapath"]
df = load_data(PREPROCESSED_DATAPATH,['_Close','_Low'])
df.head()

,Date,AIR_Low,AIR_Close,ABM_Low,ABM_Close,AEG_Low,AEG_Close,MITT_Low,MITT_Close,AMN_Low,...,XPO_Low,XPO_Close,XRX_Low,XRX_Close,XYL_Low,XYL_Close,YUM_Low,YUM_Close,ZBH_Low,ZBH_Close
0,2002-01-02,8.99,9.20,15.050,15.380,25.653847,25.903847,NaN,NaN,26.600000,...,NaN,NaN,26.666668,27.457182,NaN,NaN,8.599928,8.625090,29.106796,29.349516
1,2002-01-03,9.28,9.66,15.125,15.320,25.721153,26.000000,NaN,NaN,27.100000,...,NaN,NaN,27.009224,27.404480,NaN,NaN,8.630482,8.806614,28.932039,29.165049
2,2002-01-04,9.66,9.84,15.250,15.605,24.692308,25.038462,NaN,NaN,26.500000,...,NaN,NaN,25.639000,26.482212,NaN,NaN,8.770669,9.148095,28.640778,28.689320
3,2002-01-07,9.65,9.70,15.300,15.300,24.355768,24.576923,NaN,NaN,26.850000,...,NaN,NaN,23.978920,26.034256,NaN,NaN,9.205607,9.318835,28.466019,28.689320
4,2002-01-08,9.65,9.85,14.915,15.020,23.884615,24.269232,NaN,NaN,26.450001,...,NaN,NaN,25.032938,26.218710,NaN,NaN,9.435658,9.498562,28.601942,29.611650


In [18]:
# Size of data
num_stocks = len(df.filter(regex='Close').columns.tolist())
print(f'Number of stocks: {num_stocks}')
print(f"Duration of backtesting: 20 years from {df['Date'].iloc[0]} to {df['Date'].iloc[-1]}")

Number of stocks: 1428
Duration of backtesting: 20 years from 2002-01-02 to 2022-12-30


In [7]:
# Top 10 tickers with missing values 
missing_values = df.isnull().any().sum()
print(f'Number of tickers with missing values: {missing_values}\
      \nTop 10 tickers with missing values:')
df.isnull().sum().sort_values(ascending=False)[:10]

Number of tickers with missing values: 934      
Top 10 tickers with missing values:


CPRI_Close    2508
CPRI_Low      2508
CIVI_Close    2508
VTLE_Close    2508
VTLE_Low      2508
CIVI_Low      2508
BUI_Low       2493
BUI_Close     2493
APTV_Close    2489
APTV_Low      2489
dtype: int64

### MACD

In [5]:
# get entry exit signals, which will be used to get the trade entry and exit dates using parallel processing. 
PREPROCESSED_DATAPATH = conf.backtest_conf["data"]["preprocessed_datapath"]
macd_df = load_data(PREPROCESSED_DATAPATH,['_Close'])
macd_positions = MACD(macd_df) 
positions = macd_positions.get_entry_exit_signal()
positions

,AAIC_Entry,AAIC_Exit,AAP_Entry,AAP_Exit,AAT_Entry,AAT_Exit,AA_Entry,AA_Exit,ABB_Entry,ABB_Exit,...,XYL_Entry,XYL_Exit,X_Entry,X_Exit,YUM_Entry,YUM_Exit,ZBH_Entry,ZBH_Exit,ZTR_Entry,ZTR_Exit
Date,,,,,,,,,,,,,,,,,,,,,
2002-01-02,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2002-01-03,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2002-01-04,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2002-01-07,nan,exit_long,nan,exit_short,nan,nan,nan,exit_long,nan,exit_long,...,nan,nan,nan,exit_long,nan,exit_long,nan,exit_short,nan,exit_long
2002-01-08,nan,exit_long,nan,exit_short,nan,nan,nan,exit_long,nan,nan,...,nan,nan,nan,exit_long,nan,exit_long,nan,exit_short,nan,exit_long
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,nan,exit_short,nan,exit_long,nan,nan,nan,nan,nan,exit_short,...,nan,nan,nan,nan,nan,nan,nan,exit_long,nan,exit_short
2022-12-27,nan,nan,nan,exit_long,nan,exit_long,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2022-12-28,nan,exit_short,nan,exit_long,nan,exit_long,nan,exit_long,nan,exit_long,...,nan,exit_long,nan,nan,nan,exit_long,nan,exit_long,nan,nan


In [6]:

column_pairs = [(positions.columns[i], positions.columns[i+1]) for i in range(0, len(positions.columns), 2)]
column_pairs

[('AAIC_Entry', 'AAIC_Exit'),
 ('AAP_Entry', 'AAP_Exit'),
 ('AAT_Entry', 'AAT_Exit'),
 ('AA_Entry', 'AA_Exit'),
 ('ABB_Entry', 'ABB_Exit'),
 ('ABC_Entry', 'ABC_Exit'),
 ('ABEV_Entry', 'ABEV_Exit'),
 ('ABG_Entry', 'ABG_Exit'),
 ('ABM_Entry', 'ABM_Exit'),
 ('ABR_Entry', 'ABR_Exit'),
 ('ABT_Entry', 'ABT_Exit'),
 ('AB_Entry', 'AB_Exit'),
 ('ACCO_Entry', 'ACCO_Exit'),
 ('ACM_Entry', 'ACM_Exit'),
 ('ACN_Entry', 'ACN_Exit'),
 ('ACR_Entry', 'ACR_Exit'),
 ('ADC_Entry', 'ADC_Exit'),
 ('ADM_Entry', 'ADM_Exit'),
 ('ADX_Entry', 'ADX_Exit'),
 ('AEE_Entry', 'AEE_Exit'),
 ('AEG_Entry', 'AEG_Exit'),
 ('AEL_Entry', 'AEL_Exit'),
 ('AEM_Entry', 'AEM_Exit'),
 ('AEO_Entry', 'AEO_Exit'),
 ('AEP_Entry', 'AEP_Exit'),
 ('AER_Entry', 'AER_Exit'),
 ('AES_Entry', 'AES_Exit'),
 ('AET_Entry', 'AET_Exit'),
 ('AFB_Entry', 'AFB_Exit'),
 ('AFG_Entry', 'AFG_Exit'),
 ('AFL_Entry', 'AFL_Exit'),
 ('AFT_Entry', 'AFT_Exit'),
 ('AGCO_Entry', 'AGCO_Exit'),
 ('AGI_Entry', 'AGI_Exit'),
 ('AGM_Entry', 'AGM_Exit'),
 ('AGO_Entry', '

In [ ]:
def lazy_function(pair):
    col1,col2 = pair[0],pair[1]
    exit_position = macd_positions.get_entry_exit_position(positions[col1],positions[col2])
    return exit_position
lazy_list = []
column_pairs = [(positions.columns[i], positions.columns[i+1]) for i in range(0, len(positions.columns), 2)]
for pair in column_pairs:
    exit_position = dask.delayed(lazy_function)(pair)
    lazy_list.append(exit_position)
exit_position = dask.compute(*lazy_list)

In [ ]:
# merge into 1 dataframe and save it
tickers = [col.split('_')[0] for col in df.iloc[:, 1:].columns]
for i in range(len(exit_position)):
    exit_position[i].rename(columns=lambda x: tickers[i] + '_' + x, inplace=True)
merged_df = pd.concat(exit_position, axis=1)
merged_df.to_csv('new_macd_positions.csv') 

In [17]:
macd_positions = pd.read_csv('../data/intermediate/macd_positions.csv')
macd_positions.drop(['Unnamed: 0'], axis=1,inplace=True)
macd_positions.head()

C:\Users\User\AppData\Local\Temp\ipykernel_11060\1543702493.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255

,A_Position,A_Entry,A_Exit,AA_Position,AA_Entry,AA_Exit,AAIC_Position,AAIC_Entry,AAIC_Exit,AAP_Position,...,XYL_Exit,YUM_Position,YUM_Entry,YUM_Exit,ZBH_Position,ZBH_Entry,ZBH_Exit,ZTR_Position,ZTR_Entry,ZTR_Exit
0,long,2002-03-11,2002-03-19,long,2002-03-19,2002-03-20,long,2002-03-08,2002-03-11,long,...,2002-04-01,long,2002-02-26,2002-02-27,long,2002-04-29,2002-05-01,long,2002-03-06,2002-03-07
1,long,2002-03-21,2002-03-22,long,2002-06-20,2002-06-21,long,2002-04-12,2002-04-15,long,...,2002-05-09,long,2002-04-10,2002-04-11,long,2002-06-20,2002-06-28,long,2002-03-21,2002-03-26
2,long,2002-04-29,2002-05-01,long,2002-07-29,2002-07-30,long,2002-04-30,2002-05-01,long,...,2002-08-02,long,2002-04-23,2002-04-24,long,2002-07-26,2002-07-29,long,2002-03-26,2002-03-28
3,long,2002-06-13,2002-06-14,long,2002-09-12,2002-09-17,long,2002-05-20,2002-06-04,long,...,2002-08-21,long,2002-05-16,2002-05-20,long,2002-09-09,2002-09-19,long,2002-04-15,2002-04-17
4,long,2002-08-01,2002-08-07,long,2002-09-30,2002-10-03,long,2002-06-11,2002-06-12,long,...,2002-08-21,long,2002-06-11,2002-06-19,long,2002-09-23,2002-09-24,long,2002-07-08,2002-07-18


In [15]:
# test with 1 stock
# Ensure data is in the required format (Date is the first column, and rename Close to Ticker_Close)
def test_macd_single_stock(ticker:str,start_date:str,end_date:str):
    test_df = yf.download(ticker,start=start_date,
                    end=end_date,)
    test_close = test_df[['Close']]
    test_close['Date'] = test_close.index
    test_close.rename(columns={"Close": "MSFT_Close"},inplace=True)
    test_close = test_close[['Date','MSFT_Close']]
    macd_positions = MACD(test_close) 
    positions = macd_positions.get_entry_exit_signal()
    return macd_positions.get_entry_exit_position(positions['MSFT_Entry'],positions['MSFT_Exit'])
test_macd_single_stock('MSFT','2022-01-01','2022-04-04')

[*********************100%***********************]  1 of 1 completed


C:\Users\User\AppData\Local\Temp\ipykernel_11060\775882921.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_close['Date'] = test_close.index
C:\Users\User\AppData\Local\Temp\ipykernel_11060\775882921.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_close.rename(columns={"Close": "MSFT_Close"},inplace=True)


,Position,Entry,Exit
0,long,2022-02-28,2022-03-01
1,long,2022-03-17,2022-03-18
2,short,2022-03-08,2022-03-09


### Profits for macd strategy

In [26]:
macd_df = pd.read_csv('../data/intermediate/new_macd_positions.csv',index_col=0)
macd_df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_3864\2391550295.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,

,AIR_Position,AIR_Entry,AIR_Exit,ABM_Position,ABM_Entry,ABM_Exit,AEG_Position,AEG_Entry,AEG_Exit,MITT_Position,...,XRX_Exit,XYL_Position,XYL_Entry,XYL_Exit,YUM_Position,YUM_Entry,YUM_Exit,ZBH_Position,ZBH_Entry,ZBH_Exit
0,long,2002-03-11,2002-03-19,long,2002-03-19,2002-03-20,long,2011-03-21,2011-03-22,long,...,2002-04-01,long,2002-02-26,2002-02-27,long,2002-04-29,2002-05-01,long,2002-03-06,2002-03-07
1,long,2002-03-21,2002-03-22,long,2002-06-20,2002-06-21,long,2011-04-25,2011-04-26,long,...,2002-05-09,long,2002-04-10,2002-04-11,long,2002-06-20,2002-06-28,long,2002-03-21,2002-03-26
2,long,2002-04-29,2002-05-01,long,2002-07-29,2002-07-30,long,2011-05-11,2011-05-12,long,...,2002-08-02,long,2002-04-23,2002-04-24,long,2002-07-26,2002-07-29,long,2002-03-26,2002-03-28
3,long,2002-06-13,2002-06-14,long,2002-09-12,2002-09-17,long,2011-06-01,2011-06-15,long,...,2002-08-21,long,2002-05-16,2002-05-20,long,2002-09-09,2002-09-19,long,2002-04-15,2002-04-17
4,long,2002-08-01,2002-08-07,long,2002-09-30,2002-10-03,long,2011-06-22,2011-06-23,long,...,2002-08-21,long,2002-06-11,2002-06-19,long,2002-09-23,2002-09-24,long,2002-07-08,2002-07-18


In [7]:
test_df = macd_df.iloc[:,:3]
test_df.columns[test_df.columns.str.endswith('_Entry')][0]

'AIR_Entry'

In [28]:
profit = Profit(test_df,close_df)
profit_df = profit.calculate_profit('long')
profit.calculate_long_term_profit(profit_df,2022,2022)

1.2265999620167967

### stocktrend 

In [2]:
# load preprocessed dataset 
PREPROCESSED_DATAPATH = conf.backtest_conf["data"]["preprocessed_datapath"]
# df = load_data(PREPROCESSED_DATAPATH,['_Close'])
df = load_data(PREPROCESSED_DATAPATH,['_Close','_Low'])
df.head()

,Date,AIR_Low,AIR_Close,ABM_Low,ABM_Close,AEG_Low,AEG_Close,MITT_Low,MITT_Close,AMN_Low,...,XPO_Low,XPO_Close,XRX_Low,XRX_Close,XYL_Low,XYL_Close,YUM_Low,YUM_Close,ZBH_Low,ZBH_Close
0,2002-01-02,8.99,9.20,15.050,15.380,25.653847,25.903847,NaN,NaN,26.600000,...,NaN,NaN,26.666668,27.457182,NaN,NaN,8.599928,8.625090,29.106796,29.349516
1,2002-01-03,9.28,9.66,15.125,15.320,25.721153,26.000000,NaN,NaN,27.100000,...,NaN,NaN,27.009224,27.404480,NaN,NaN,8.630482,8.806614,28.932039,29.165049
2,2002-01-04,9.66,9.84,15.250,15.605,24.692308,25.038462,NaN,NaN,26.500000,...,NaN,NaN,25.639000,26.482212,NaN,NaN,8.770669,9.148095,28.640778,28.689320
3,2002-01-07,9.65,9.70,15.300,15.300,24.355768,24.576923,NaN,NaN,26.850000,...,NaN,NaN,23.978920,26.034256,NaN,NaN,9.205607,9.318835,28.466019,28.689320
4,2002-01-08,9.65,9.85,14.915,15.020,23.884615,24.269232,NaN,NaN,26.450001,...,NaN,NaN,25.032938,26.218710,NaN,NaN,9.435658,9.498562,28.601942,29.611650


In [3]:
df= df.iloc[:,:51]
df

,Date,AIR_Low,AIR_Close,ABM_Low,ABM_Close,AEG_Low,AEG_Close,MITT_Low,MITT_Close,AMN_Low,...,AEM_Low,AEM_Close,AL_Low,AL_Close,ALG_Low,ALG_Close,ALK_Low,ALK_Close,ALB_Low,ALB_Close
0,2002-01-02,8.990000,9.200000,15.050000,15.380000,25.653847,25.903847,NaN,NaN,26.600000,...,9.830000,9.890000,NaN,NaN,14.250000,14.250000,7.025000,7.242500,11.655000,11.950000
1,2002-01-03,9.280000,9.660000,15.125000,15.320000,25.721153,26.000000,NaN,NaN,27.100000,...,9.830000,9.910000,NaN,NaN,13.500000,13.600000,7.212500,7.500000,11.880000,12.095000
2,2002-01-04,9.660000,9.840000,15.250000,15.605000,24.692308,25.038462,NaN,NaN,26.500000,...,9.850000,9.920000,NaN,NaN,13.720000,14.000000,7.507500,7.972500,12.075000,12.210000
3,2002-01-07,9.650000,9.700000,15.300000,15.300000,24.355768,24.576923,NaN,NaN,26.850000,...,9.900000,10.170000,NaN,NaN,14.100000,14.800000,7.802500,7.837500,11.950000,11.950000
4,2002-01-08,9.650000,9.850000,14.915000,15.020000,23.884615,24.269232,NaN,NaN,26.450001,...,10.000000,10.000000,NaN,NaN,14.100000,14.100000,7.800000,7.887500,11.700000,11.950000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5282,2022-12-23,42.200001,42.889999,43.369999,44.110001,5.110000,5.140000,5.66,5.76,101.849998,...,51.299999,52.619999,36.580002,37.360001,138.149994,141.529999,42.610001,43.310001,221.240005,226.960007
5283,2022-12-27,43.099998,44.310001,43.509998,44.590000,5.120000,5.140000,5.64,5.69,101.440002,...,52.580002,53.540001,37.240002,38.130001,140.770004,141.110001,41.880001,42.259998,213.520004,214.160004
5284,2022-12-28,43.599998,43.689999,43.959999,44.090000,5.030000,5.040000,5.43,5.45,101.510002,...,51.930000,52.029999,37.259998,37.290001,140.350006,141.360001,41.290001,41.299999,210.419998,214.039993
5285,2022-12-29,43.950001,44.700001,43.990002,44.770000,5.060000,5.090000,5.18,5.37,102.879997,...,52.080002,52.150002,37.410000,38.340000,140.669998,143.279999,41.410000,42.810001,216.869995,217.470001


### Parallel processing

In [4]:
parallel_processing = ParallelProcess(df,STRATEGY)
# parallel_processing.run_pipeline()
parallel_processing.prepare_data(4)

            Date    AIR_Low  AIR_Close    ABM_Low  ABM_Close    AEG_Low  \
0     2002-01-02   8.990000   9.200000  15.050000  15.380000  25.653847   
1     2002-01-03   9.280000   9.660000  15.125000  15.320000  25.721153   
2     2002-01-04   9.660000   9.840000  15.250000  15.605000  24.692308   
3     2002-01-07   9.650000   9.700000  15.300000  15.300000  24.355768   
4     2002-01-08   9.650000   9.850000  14.915000  15.020000  23.884615   
...          ...        ...        ...        ...        ...        ...   
5282  2022-12-23  42.200001  42.889999  43.369999  44.110001   5.110000   
5283  2022-12-27  43.099998  44.310001  43.509998  44.590000   5.120000   
5284  2022-12-28  43.599998  43.689999  43.959999  44.090000   5.030000   
5285  2022-12-29  43.950001  44.700001  43.990002  44.770000   5.060000   
5286  2022-12-30  44.400002  44.900002  44.189999  44.419998   5.010000   

      AEG_Close  MITT_Low  MITT_Close     AMN_Low  ...    AEM_Low  AEM_Close  \
0     25.903847    

UnboundLocalError: local variable 'partitioned_df_list' referenced before assignment

In [18]:
exit_position_list

,AAP_Position,AAP_Entry,AAP_Exit,AAP_Position,AAP_Entry,AAP_Exit,AAT_Position,AAT_Entry,AAT_Exit,AAT_Position,...,AXP_Exit,AXP_Position,AXP_Entry,AXP_Exit,PEO_Position,PEO_Entry,PEO_Exit,PEO_Position,PEO_Entry,PEO_Exit
0,long,2002-03-19,2002-03-20,long,2011-03-21,2011-03-22,long,2002-05-10,2002-05-13,long,...,2002-04-22,long,2002-03-05,2002-03-06,long,2002-05-30,2002-06-18,long,2002-05-03,2002-05-09
1,long,2002-06-20,2002-06-21,long,2011-04-25,2011-04-26,long,2002-06-27,2002-06-28,long,...,2002-05-31,long,2002-06-04,2002-06-06,long,2002-06-26,2002-07-29,long,2002-07-08,2002-07-09
2,long,2002-07-29,2002-07-30,long,2011-05-11,2011-05-12,long,2002-08-01,2002-08-09,long,...,2002-07-25,long,2002-07-29,2002-08-09,long,2002-07-10,2002-07-29,long,2002-08-01,2002-08-02
3,long,2002-09-12,2002-09-17,long,2011-06-01,2011-06-15,long,2002-08-07,2002-08-09,long,...,2002-10-18,long,2002-08-09,2002-08-15,long,2002-07-31,2002-08-01,long,2002-09-13,2002-10-04
4,long,2002-09-30,2002-10-03,long,2011-06-22,2011-06-23,long,2002-10-02,2002-10-14,long,...,2002-12-05,long,2002-10-09,2002-10-14,long,2002-10-16,2002-10-17,long,2002-10-11,2002-10-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# rearrange 'Date' column to be the first column 
rearranged_df = pd.concat([df['Date'], df.drop('Date', axis=1)], axis=1)
rearranged_df = rearranged_df.iloc[:,:51]
rearranged_df.head()

,Date,AIR_Low,AIR_Close,ABM_Low,ABM_Close,AEG_Low,AEG_Close,MITT_Low,MITT_Close,AMN_Low,...,AEM_Low,AEM_Close,AL_Low,AL_Close,ALG_Low,ALG_Close,ALK_Low,ALK_Close,ALB_Low,ALB_Close
0,2002-01-02,8.99,9.20,15.050,15.380,25.653847,25.903847,NaN,NaN,26.600000,...,9.83,9.89,NaN,NaN,14.25,14.25,7.0250,7.2425,11.655,11.950
1,2002-01-03,9.28,9.66,15.125,15.320,25.721153,26.000000,NaN,NaN,27.100000,...,9.83,9.91,NaN,NaN,13.50,13.60,7.2125,7.5000,11.880,12.095
2,2002-01-04,9.66,9.84,15.250,15.605,24.692308,25.038462,NaN,NaN,26.500000,...,9.85,9.92,NaN,NaN,13.72,14.00,7.5075,7.9725,12.075,12.210
3,2002-01-07,9.65,9.70,15.300,15.300,24.355768,24.576923,NaN,NaN,26.850000,...,9.90,10.17,NaN,NaN,14.10,14.80,7.8025,7.8375,11.950,11.950
4,2002-01-08,9.65,9.85,14.915,15.020,23.884615,24.269232,NaN,NaN,26.450001,...,10.00,10.00,NaN,NaN,14.10,14.10,7.8000,7.8875,11.700,11.950


In [8]:
# implement stocktrend strategy using dask parallel processing for all stocks 
import dask
def lazy_function(stock):
    stocktrend = StockTrend(rearranged_df[stock])
    stocktrend.generate_buy_dates()
    stocktrend_exit_df = stocktrend.generate_sell_dates()
    return stocktrend_exit_df
lazy_list = []
stock_columns = [['Date', rearranged_df.columns[i], rearranged_df.columns[i+1]]for i in range(1, len(rearranged_df.columns[1:]), 2)]
for stock in stock_columns:
    exit_position = dask.delayed(lazy_function)(stock)
    lazy_list.append(exit_position)
exit_position = dask.compute(*lazy_list)

c:\Users\User\Desktop\side projects\backtesting\notebooks\../src\strategies\stock_trend_following_system.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  def generate_buy_dates(self):
c:\Users\User\Desktop\side projects\backtesting\notebooks\../src\strategies\stock_trend_following_system.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # 1: close price above 100MA, 0: close price below 100MA
c:\Users\User\Desktop\side projects\backtesting\notebooks\../src\strategies\stock_trend_following_system.py:3

In [8]:
# merge into 1 dataframe and save it
tickers = [col.split('_')[0] for col in rearranged_df.iloc[:, 1::2].columns]
for i in range(len(exit_position)):
    exit_position[i].rename(columns=lambda x: tickers[i] + '_' + x, inplace=True)
merged_df = pd.concat(exit_position, axis=1)
merged_df
# merged_df.to_csv('stocktrend_positions.csv') 

,AIR_entry_date,AIR_exit_date,ABM_entry_date,ABM_exit_date,AEG_entry_date,AEG_exit_date,MITT_entry_date,MITT_exit_date,AMN_entry_date,AMN_exit_date,...,RFI_entry_date,RFI_exit_date,CL_entry_date,CL_exit_date,STK_entry_date,STK_exit_date,CMA_entry_date,CMA_exit_date,CMC_entry_date,CMC_exit_date
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5282,,,,,,,2022-12-28,2022-12-30,,,...,,,,,,,,,,
5283,,,,,,,2022-12-28,2022-12-30,,,...,,,,,,,,,,
5284,,,,,,,2022-12-28,2022-12-30,,,...,,,,,,,,,,
5285,,,,,,,,,,,...,,,,,,,,,,


In [5]:
test_df = rearranged_df.iloc[:,:3]
test_df

,Date,AIR_Low,AIR_Close
0,2002-01-02,8.990000,9.200000
1,2002-01-03,9.280000,9.660000
2,2002-01-04,9.660000,9.840000
3,2002-01-07,9.650000,9.700000
4,2002-01-08,9.650000,9.850000
...,...,...,...
5282,2022-12-23,42.200001,42.889999
5283,2022-12-27,43.099998,44.310001
5284,2022-12-28,43.599998,43.689999
5285,2022-12-29,43.950001,44.700001


In [6]:
stocktrend = StockTrend(test_df)
stocktrend.generate_buy_dates()
stocktrend_exit_df = stocktrend.generate_sell_dates()
stocktrend_exit_df

TypeError: Cannot index by location index with a non-integer key

### Golden death cross

In [7]:
golden_death_cross = GoldenDeathCross(df)
output = golden_death_cross.get_cross(df['A_Close'])
output[output['cross']=='golden']

,Date,cross,long_term_ma
351,2003-05-27,golden,10.832940
880,2005-06-30,golden,16.508977
1243,2006-12-07,golden,24.061051
1632,2008-06-26,golden,24.671781
1890,2009-07-07,golden,13.468348
2223,2010-10-29,golden,22.533691
2556,2012-02-27,golden,28.641774
2778,2013-01-15,golden,28.082582
3268,2014-12-24,golden,40.388131
3333,2015-03-31,golden,40.602784


### Optimising MACD strategy

In [6]:
# timeit
import timeit
mysetup = """
import pandas as pd
import sys
sys.path.append('../src')
from data import load_preprocessed_data, load_close_data
from strategies import MACD
from hydra import initialize, compose
import numpy as np
with initialize(version_base=None, config_path="../conf"):
    datapipeline_cfg = compose(config_name='datapipeline')
stock_data = load_close_data(datapipeline_cfg)
df = stock_data.copy()
macd_positions = MACD(df) 
positions = macd_positions.apply()
"""
mycode = '''
columns = positions.columns
for i in range(0, len(columns), 2):
    col1 = columns[i]
    col2 = columns[i + 1]
    col_name = f'{col1.split("_")[0]}_Exit_Position'
    exit_position = macd_positions.get_exit_position(positions[col1],positions[col2]).rename(col_name)
    positions = pd.concat([positions, exit_position], axis=1)
'''
print(timeit.timeit(setup=mysetup,
                    stmt=mycode,
                    number=1))

1050.2912589


### Get macd entry and exit positions and save as csv file
Dask parallel processing saved about 4min 

In [12]:
import dask
def lazy_function(pair):
    col1,col2 = pair[0],pair[1]
    exit_position = macd_positions.get_entry_exit_position(positions[col1],positions[col2])
    return exit_position
lazy_list = []
column_pairs = [(positions.columns[i], positions.columns[i+1]) for i in range(0, len(positions.columns), 2)]
for pair in column_pairs:
    exit_position = dask.delayed(lazy_function)(pair)
    lazy_list.append(exit_position)
exit_position = dask.compute(*lazy_list)

In [13]:
# merge into 1 dataframe and save it
tickers = [col.split('_')[0] for col in df.iloc[:, 1:].columns]
for i in range(len(exit_position)):
    exit_position[i].rename(columns=lambda x: tickers[i] + '_' + x, inplace=True)
merged_df = pd.concat(exit_position, axis=1)
merged_df.to_csv('new_macd_positions.csv') 

In [51]:
from dask.dataframe import from_pandas
# ddf = from_pandas(positions, npartitions=1425)

def lazy_function(partition):
    col1,col2 = partition.columns[0],partition.columns[1]
    exit_position = macd_positions.get_exit_position(partition[col1],partition[col2])
    return exit_position

# Apply split_dataframe_by_columns to each partition
partitions = [positions.iloc[:, i:i+2] for i in range(0, positions.shape[1], 2)]
ddf_partitions = [from_pandas(partition, npartitions=1) for partition in partitions]
result = [partition.map_partitions(lazy_function).compute() for partition in ddf_partitions]
# result = new_df.compute()
print(result)


KeyboardInterrupt: 

In [50]:
import multiprocessing as mp
pool = mp.Pool()

def lazy_function(pair):
    col1,col2 = pair[0],pair[1]
    exit_position = macd_positions.get_entry_exit_position(positions[col1],positions[col2])
    return exit_position
lazy_list = []
column_pairs = [(positions.columns[i], positions.columns[i+1]) for i in range(0, len(positions.columns), 2)]
for pair in column_pairs:
    result = pool.apply_async(lazy_function, args=(pair,))
    lazy_list.append(result)
pool.close()
pool.join()

[     AAIC_Entry   AAIC_Exit
 0           nan         nan
 1           nan         nan
 2           nan         nan
 3           nan   exit_long
 4           nan   exit_long
 ...         ...         ...
 5282        nan  exit_short
 5283        nan         nan
 5284        nan  exit_short
 5285        nan  exit_short
 5286        nan   exit_long
 
 [5287 rows x 2 columns],
      AAP_Entry    AAP_Exit
 0          nan         nan
 1          nan         nan
 2          nan         nan
 3          nan  exit_short
 4          nan  exit_short
 ...        ...         ...
 5282       nan   exit_long
 5283       nan   exit_long
 5284       nan   exit_long
 5285       nan   exit_long
 5286       nan   exit_long
 
 [5287 rows x 2 columns],
      AAT_Entry    AAT_Exit
 0          nan         nan
 1          nan         nan
 2          nan         nan
 3          nan   exit_long
 4          nan  exit_short
 ...        ...         ...
 5282       nan         nan
 5283       nan         nan
 5284   

In [14]:
import concurrent.futures
def lazy_function(pair):
    col1,col2 = pair[0],pair[1]
    exit_position = macd_positions.get_entry_exit_position(positions[col1],positions[col2])
    return exit_position
column_pairs = [(positions.columns[i], positions.columns[i+1]) for i in range(0, len(positions.columns), 2)]
with concurrent.futures.ProcessPoolExecutor() as executor:
    # can replace ProcessPoolExecutor with ThreadPoolExecutor
    result = list(executor.map(lazy_function, [pair for pair in column_pairs]))

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [31]:
# convert to dask df
from dask.dataframe import from_pandas
ddf = from_pandas(positions, npartitions=2)
column_pairs = [(ddf.columns[i], ddf.columns[i+1]) for i in range(0, len(ddf.columns), 2)]
for pair in column_pairs:
pd.concat([positions, exit_position], axis=1)
